# Loading Yelp's Data and Processing

## Import the packages

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

%matplotlib inline
plt.style.use('default')
print(plt.style.available)
#print all available styles

print (pd.__version__)

import os
from IPython.core.display import HTML

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']
0.20.3


In [2]:
!echo

ECHO is on.


# Yelp Dataset Challenge URL

This link gives you an idea of the structure of Data Schema: all JSON files with each line as a distinct record
https://www.yelp.com/dataset/challenge

Except dataset_photo.json, there are 5 tables:

1. Business
2. Review
3. Checkins
4. Tips
5. Users

Further down this notebook, we will see that the Reviews table is a 1GB file that's not easy to load directly into IPython RAM, so we'll chop it up into 4 pieces along the way of our cleaning process.

In [3]:
YELP_DATACHALLENGE_URL = "https://www.yelp.com/dataset/challenge"
tag = '<iframe src = "{0}" width = "1000px" height = "333px">'.format(YELP_DATACHALLENGE_URL)
HTML(tag)

## Read data from file and load to Pandas DataFrame

In [4]:
file_business, file_checkin, file_tip, file_user, file_review = [
    'yelp_academic_dataset_business.json',
    'yelp_academic_dataset_checkin.json',
    'yelp_academic_dataset_tip.json',
    'yelp_academic_dataset_user.json',
    'yelp_academic_dataset_review.json'
]

### Business Data

In [5]:
# open file
f = open(file_business,encoding="utf8")
first_line = f.readline()
first_line_json = json.loads(first_line)

print (type(first_line))
print (first_line)
print (type(first_line_json))
print (first_line_json)

<class 'str'>
{"business_id":"Apn5Q_b6Nz61Tq4XzPdf9A","name":"Minhas Micro Brewery","neighborhood":"","address":"1314 44 Avenue NE","city":"Calgary","state":"AB","postal_code":"T2E 6L6","latitude":51.0918130155,"longitude":-114.031674872,"stars":4.0,"review_count":24,"is_open":1,"attributes":{"BikeParking":"False","BusinessAcceptsCreditCards":"True","BusinessParking":"{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}","GoodForKids":"True","HasTV":"True","NoiseLevel":"average","OutdoorSeating":"False","RestaurantsAttire":"casual","RestaurantsDelivery":"False","RestaurantsGoodForGroups":"True","RestaurantsPriceRange2":"2","RestaurantsReservations":"True","RestaurantsTakeOut":"True"},"categories":"Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel","hours":{"Monday":"8:30-17:0","Tuesday":"11:0-21:0","Wednesday":"11:0-21:0","Thursday":"11:0-21:0","Friday":"11:0-21:0","Saturday":"11:0-21:0"}}

<class 'dict'>
{'business_id': 'Apn5Q_b6Nz61Tq4XzPdf9A'

In [6]:
with open(file_business,encoding="utf8") as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [7]:
df_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [8]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


### Checkin Data

In [9]:
with open(file_checkin,encoding="utf8") as f:
    df_checkin = pd.DataFrame(json.loads(line) for line in f)

In [10]:
df_checkin.head()

,business_id,time
0,7KPBkxAOEtb3QeIL9PEErg,"{'Fri-0': 2, 'Sat-0': 1, 'Sun-0': 1, 'Wed-0': ..."
1,kREVIrSBbtqBhIYkTccQUg,"{'Mon-13': 1, 'Thu-13': 1, 'Sat-16': 1, 'Wed-1..."
2,tJRDll5yqpZwehenzE2cSg,"{'Thu-0': 1, 'Mon-1': 1, 'Mon-12': 1, 'Sat-16'..."
3,tZccfdl6JNw-j5BKnCTIQQ,"{'Sun-14': 1, 'Fri-18': 1, 'Mon-20': 1}"
4,r1p7RAMzCV_6NPF0dNoR3g,"{'Sat-3': 1, 'Sun-18': 1, 'Sat-21': 1, 'Sat-23..."


In [11]:
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157075 entries, 0 to 157074
Data columns (total 2 columns):
business_id    157075 non-null object
time           157075 non-null object
dtypes: object(2)
memory usage: 2.4+ MB


### Tip Data

In [12]:
with open(file_tip,encoding="utf8") as f:
    df_tip = pd.DataFrame(json.loads(line) for line in f)
    
df_tip.head(2)
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185348 entries, 0 to 1185347
Data columns (total 5 columns):
business_id    1185348 non-null object
date           1185348 non-null object
likes          1185348 non-null int64
text           1185348 non-null object
user_id        1185348 non-null object
dtypes: int64(1), object(4)
memory usage: 45.2+ MB


### User Data

In [13]:
with open(file_user,encoding="utf8") as f:
    df_user = pd.DataFrame(json.loads(line) for line in f)
    
df_user.head(2)
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518169 entries, 0 to 1518168
Data columns (total 22 columns):
average_stars         1518169 non-null float64
compliment_cool       1518169 non-null int64
compliment_cute       1518169 non-null int64
compliment_funny      1518169 non-null int64
compliment_hot        1518169 non-null int64
compliment_list       1518169 non-null int64
compliment_more       1518169 non-null int64
compliment_note       1518169 non-null int64
compliment_photos     1518169 non-null int64
compliment_plain      1518169 non-null int64
compliment_profile    1518169 non-null int64
compliment_writer     1518169 non-null int64
cool                  1518169 non-null int64
elite                 1518169 non-null object
fans                  1518169 non-null int64
friends               1518169 non-null object
funny                 1518169 non-null int64
name                  1518169 non-null object
review_count          1518169 non-null int64
useful                15181

### Review Data

In [14]:
with open(file_review,encoding="utf8") as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
    
df_review.head(2)
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5996996 entries, 0 to 5996995
Data columns (total 9 columns):
business_id    object
cool           int64
date           object
funny          int64
review_id      object
stars          int64
text           object
useful         int64
user_id        object
dtypes: int64(4), object(5)
memory usage: 411.8+ MB
